# Visual Geometry Group

    Has Multiple Conv and Pooling layers in sequence
    Dropout and RelU added so it makes sense
    Paper doesn't have dropout layer
    In and Out channels remains same between Maxpools

In [1]:
import torch
import torch.nn as nn

In [28]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "VGG19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}


In [60]:
class VGG(nn.Module):
    def __init__(self,in_channels = 3,num_classes = 1000):
        super(VGG,self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_layers(VGG_types["VGG19"])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
        
    def forward(self,x):
        x = self.conv_layers(x)

        x = x.reshape(x.shape[0], -1)
        
        x = self.fcs(x)
        
        return x

    def create_layers(self, arch):
        layers = []
        in_channels = self.in_channels

        for x in arch:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]

                in_channels = x
            
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

In [61]:
BATCH_SIZE = 5
x = torch.randn(BATCH_SIZE, 3, 224, 224)
model = VGG()
print(model(x).shape)
assert model(x).shape == torch.Size([BATCH_SIZE,1000])

torch.Size([5, 1000])
